In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import pprint
import keras.backend as KB
sys.path.append('../')

import mrcnn.model        as modellib
import mrcnn.visualize    as visualize
# import mrcnn.new_shapes   as shapes
import mrcnn.new_shapes  as new_shapes
# from mrcnn.new_shapes  import NewShapesDataset, NewShapesConfig
from mrcnn.config      import Config
from mrcnn.model       import log
from mrcnn.dataset     import Dataset 
# from mrcnn.pc_prototype import PCTensor
# from mrcnn.pcn_layer    import PCNLayer, PCILayer

# Root directory of the project
ROOT_DIR = os.getcwd()
MODEL_PATH = 'E:\Models'
# Directory to save logs and trained model
MODEL_DIR = os.path.join(MODEL_PATH, "mrcnn_logs")
# Path to COCO trained weights
COCO_MODEL_PATH   = os.path.join(MODEL_PATH, "mask_rcnn_coco.h5")
RESNET_MODEL_PATH = os.path.join(MODEL_PATH, "resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5")

print("Tensorflow Version: {}   Keras Version : {} ".format(tf.__version__,keras.__version__))
pp = pprint.PrettyPrinter(indent=2, width=100)
np.set_printoptions(linewidth=100)

# Build configuration object -----------------------------------------------
config = new_shapes.NewShapesConfig()
config.BATCH_SIZE      = 2                    #Batch size is 2 (# GPUs * images/GPU).
config.IMAGES_PER_GPU  = 2
config.STEPS_PER_EPOCH = 7
config.IMAGES_PER_GPU  = 1
config.display() 

# Build shape dataset        -----------------------------------------------

# from mrcnn.datagen import data_generator, load_image_gt

# Training dataset generate 500 shapes 
dataset_test = new_shapes.NewShapesDataset()
dataset_test.load_shapes(500, config.IMAGE_SHAPE[0], config.IMAGE_SHAPE[1])
dataset_test.prepare()

import pprint
pp = pprint.PrettyPrinter(indent=4)

class InferenceConfig(new_shapes.NewShapesConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()
inference_config.display() 

# Configurations

# import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
# tf_config = tf.ConfigProto()
# tf_config.gpu_options.per_process_gpu_memory_fraction = 0.55
# set_session(tf.Session(config=tf_config))

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.55)
set_session(tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)))

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)
print(' Build complete')

#### List trainable layers using `get_trainable_layers()`

#### List trainable layers using `get_trainable_layers()` 

In [1]:
for i in  mrcnn_model.get_trainable_layers():
    print(' Layer:', i.name)     

SyntaxError: invalid syntax (<ipython-input-1-bc861564811e>, line 2)

#### 

In [ ]:
# for layer in model.keras_model.layers
#     print layer.name 
mdl = model.keras_model
layers    = [layer for layer in mdl.layers]          # all layer outputs
len(layers)

In [ ]:
idx = 1
print(' layer {} :  {}'.format(idx,layers[idx]))
pp.pprint(len(layers[idx]._inbound_nodes))
pp.pprint(layers[idx].__dict__)
print('-------------------------------------------')

In [ ]:
for idx,layer in enumerate(layers):
    print('>layer {} : name : {:40s}  type: {}'.format(idx,layer.name,layer))
#     pp.pprint(layer._inbound_nodes[0].__dict__)

In [ ]:
for idx,layer in enumerate(layers):
    print('>layer {} : name : {}  type: {}'.format(idx,layer.name,layer))
#     pp.pprint(layer._inbound_nodes[0].__dict__)
    print(">> input_shape {} inbound node {} ".format(layer._inbound_nodes[0].input_shapes,layer._inbound_nodes[0]))
#     print('>> inbound node output_shape {}'.format(layer._inbound_nodes[0].output_shapes))
#     if isinstance("layer", mrcnn.model.BatchNorm):
#         print('Batch Nomraliation layer')
    if 'padding' in layer.__dict__:
        print('\t padding : ',layer.padding)
    if 'filters' in layer.__dict__:
        print('\t Filters : ',layer.filters, ' Kernel Size :', layer.kernel_size, 'Stride:', layer.strides, ' Shape',layer.kernel.shape)
    
    if 'activation' in layer.__dict__:
        print('\t ',layer.activation)
    if 'pool_size' in layer.__dict__:
        print('\t poolsize : ',layer.pool_size,'Stride:', layer.strides)

#     pp.pprint(layer.__dict__)
    print("<< outbound shape{} input_shape {}".format(layer._outbound_nodes[0].input_shapes,layer._outbound_nodes[0]))
#     print('<< outbound node output_shape {}'.format(layer._outbound_nodes[0].output_shapes))
    print('-------------------------------------------')
    #layer.name, layer.input._keras_shape, layer.output._keras_shap

In [ ]:
# for layer in model.keras_model.layers
#     print layer.name 
mdl = model.keras_model
print(' Total number of layers ', len(model.keras_model.layers), type(mdl))

print('\n Inputs')    
for i, inp in enumerate(mdl.input):
    print(' {}   {:35s}  {}  '.format(i, inp.name, inp.shape))
    
print('\n Outputs')    
for i, outp in enumerate(mdl.output):
    print(' {}   {:35s}  {}  '.format(i, outp.name, outp.shape))


# pp = pprint.PrettyPrinter()
for idx,layer in enumerate(mdl.layers): 
    if idx > 25  : 
        break
    print()    
    print('#'*100)
    print('#  Layer {} :  Name : {}  Type: {}'.format(idx,layer.name,layer))
    print('#'*100)
    if isinstance(layer, keras.engine.training.Model):
        print('#  MODEL LAYER ')    
#         print('\n Inputs')    
#         for i, inp in enumerate(layer.input):
#             print(' {}   {:35s}  {}  '.format(i, inp.name, inp.shape))

#         print('\n Outputs')    
#         for i, outp in enumerate(layer.output):
#             print(' {}   {:35s}  {}  '.format(i, outp.name, outp.shape))
            
    pp.pprint(layer.__dict__)
    num_inbound  = len(layer._inbound_nodes)
    print('  >>>','-'*85)   
    print('  >>>  Number of inbound nodes ', num_inbound)    
    print('  >>>','-'*85)   
    if num_inbound > 0:

        for i, ib_node in enumerate(layer._inbound_nodes):
            print('\t','-'*70)
            print("\t >> {}  {}  ".format(i, ib_node))            
            for j in ib_node.input_tensors:
                print('\t    incoming tensor(s): {:25s}  shape {}'.format(j.name, j.shape))
            print('\t','-'*70)
                
            pp.pprint(ib_node.__dict__, indent=10, width=140)
    #       print('<< outbound node output_shape {}'.format(layer._outbound_nodes[0].output_shapes))
    
    

#     if isinstance("layer", model.Keras_BatchNorm):
#         print('Batch Nomraliation layer')
    if 'padding' in layer.__dict__:
        print('\t padding : ',layer.padding)
    if 'filters' in layer.__dict__:
        print('\t Filters : ',layer.filters, ' Kernel Size :', layer.kernel_size, 'Stride:', layer.strides, ' Shape',layer.kernel.shape)
    
    if 'activation' in layer.__dict__:
        print('\t ',layer.activation)
    if 'pool_size' in layer.__dict__:
        print('\t poolsize : ',layer.pool_size,'Stride:', layer.strides)
        
    num_outbound = len(layer._outbound_nodes)
    print('  <<<','-'*85)       
    print('  <<<   Number of outbound nodes ', num_outbound)
    print('  <<<','-'*85)       
#     pp.pprint(layer.__dict__)
    if num_outbound > 0:

        for i, ob_node in enumerate(layer._outbound_nodes):
            print("<< outbound {} input_shape {}".format(i,ob_node.input_shapes))
            for j in ob_node.input_tensors:
                print(' from layer: ', j.name, j.shape)
    #         for j in ob_node.outbound_layer:
    #             print(' to layer: ',j.name)
            pp.pprint(ob_node.__dict__, indent=10, width=140)
    #       print('<< outbound node output_shape {}'.format(layer._outbound_nodes[0].output_shapes))
        
    #layer.name, layer.input._keras_shape, layer.output._keras_shape

In [ ]:
# for layer in model.keras_model.layers
#     print layer.name 

layers    = [layer for layer in mrcnn_model.keras_model.layers]          # all layer outputs
print(len(layers))

for idx in range(200, len(layers)):
    print( ' layer: {} name: {}'.format(idx, layers[idx].name))
#     print(' layer {} :  {}'.format(idx,layers[idx]))
#     print(' Number of inbound nodes: ', end ='')
#     pp.pprint(len(layers[idx]._inbound_nodes))
#     pp.pprint(layers[idx].__dict__)
    print('-------------------------------------------')

In [4]:
print('idx : ', idx)
pp.pprint(layers[idx].__dict__)

print()
pp.pprint(layers[idx]._inbound_nodes[0].__dict__)

377
 layer: 200 name: add_19
-------------------------------------------
 layer: 201 name: res4l_out
-------------------------------------------
 layer: 202 name: res4m_branch2a
-------------------------------------------
 layer: 203 name: bn4m_branch2a
-------------------------------------------
 layer: 204 name: activation_40
-------------------------------------------
 layer: 205 name: res4m_branch2b
-------------------------------------------
 layer: 206 name: bn4m_branch2b
-------------------------------------------
 layer: 207 name: activation_41
-------------------------------------------
 layer: 208 name: res4m_branch2c
-------------------------------------------
 layer: 209 name: bn4m_branch2c
-------------------------------------------
 layer: 210 name: add_20
-------------------------------------------
 layer: 211 name: res4m_out
-------------------------------------------
 layer: 212 name: res4n_branch2a
-------------------------------------------
 layer: 213 name: bn4n_bra

####  Print Layer & Weight Info

In [13]:
# keras_model = mrcnn_model.keras_model
# layers = keras_model.inner_model.layers if hasattr(keras_model, "inner_model") else keras_model.layers
# print(dir(layers[0]))
# pp.pprint(layers[7].weights)
print()
for i in range(len(layers)):
    print('\n{:3d} {:20s} Weights: {}'.format(i, layers[i].name, len(layers[i].weights))) ## [weight.name for weight in layers[i].weights]))
    print('{:23s}  Trainable: {} '.format(' ',[weight.name for weight in layers[i].trainable_weights]))
    for weight in layers[i].weights:
        print('{:25s} {:30s} {}  '.format(' ',weight.name,  weight.shape))



  0 input_image          Weights: 0
                         Trainable: [] 

  1 zero_padding2d_1     Weights: 0
                         Trainable: [] 

  2 conv1                Weights: 2
                         Trainable: ['conv1/kernel:0', 'conv1/bias:0'] 
                          conv1/kernel:0                 (7, 7, 3, 64)  
                          conv1/bias:0                   (64,)  

  3 bn_conv1             Weights: 4
                         Trainable: ['bn_conv1/gamma:0', 'bn_conv1/beta:0'] 
                          bn_conv1/gamma:0               (64,)  
                          bn_conv1/beta:0                (64,)  
                          bn_conv1/moving_mean:0         (64,)  
                          bn_conv1/moving_variance:0     (64,)  

  4 activation_1         Weights: 0
                         Trainable: [] 

  5 max_pooling2d_1      Weights: 0
                         Trainable: [] 

  6 res2a_branch2a       Weights: 2
                         Trainabl

####  More layer information

In [14]:
import pprint
pp = pprint.PrettyPrinter(indent=2, width=100)
np.set_printoptions(linewidth=100,precision=4,threshold=1000, suppress = True)

# model.compile_only(learning_rate=config.LEARNING_RATE, layers='heads')
# print('\n Metrics (_get_deduped_metrics_names():) ') 
# pp.pprint(mm._get_deduped_metrics_names())
# print('\n Losses (model.metrics_names): ') 
# pp.pprint(mm.metrics_names)
# KB.set_learning_phase(0)
# print(' Learning phase values is L ' ,KB.learning_phase())
# print('\Layers ', len(mrcnn_model.keras_model.layers))
# pp.pprint(mrcnn_model.keras_model.layers)

print('\n Weights: ') 
## Get weieights acturally returns weight dims
# print(' Number of weights arrays: ',len(mrcnn_model.keras_model.get_weights()))
print(' length of model.keras_model.weights', len(mrcnn_model.keras_model.weights))
# for i,j in enumerate(mrcnn_model.keras_model.weights):
#     print(i, j) # j.name, j.shape, j.dtype)
# pp.pprint(mrcnn_model.keras_model.weights)
# pp.pprint(type(mrcnn_model.keras_model.get_weights()))

# pp.pprint(dir(model.keras_model))
# print(mrcnn_model.keras_model.weights[1])


 Inputs: 
[ <tf.Tensor 'input_image:0' shape=(?, 1024, 1024, 3) dtype=float32>,
  <tf.Tensor 'input_image_meta:0' shape=(?, ?) dtype=float32>]

 Outputs: 
[ <tf.Tensor 'mrcnn_detection/PyFunc:0' shape=<unknown> dtype=float32>,
  <tf.Tensor 'ROI/rpn_roi_proposals:0' shape=(1, ?, ?) dtype=float32>,
  <tf.Tensor 'mrcnn_class/Reshape_1:0' shape=(?, 1000, 81) dtype=float32>,
  <tf.Tensor 'mrcnn_bbox/Reshape:0' shape=(?, 1000, 81, 4) dtype=float32>]

 Weights: 
 length of model.keras_model.weights 662


#### Loaded dataset information

In [16]:
print("Testing Dataset Image Count: {}".format(len(dataset_test.image_ids)))
print("Testing Dataset Class Count: {}".format(dataset_test.num_classes))
# for i, info in enumerate(dataset_test.class_info):
#     print("{:3}. {:50}".format(i, info['name']))

Testing Dataset Image Count: 40137
Testing Dataset Class Count: 81


#### List trainable layers using `get_trainable_layers()`

In [17]:
for i in  mrcnn_model.get_trainable_layers():
    print(' Layer:', i.name)       

 Layer: conv1
 Layer: bn_conv1
 Layer: res2a_branch2a
 Layer: bn2a_branch2a
 Layer: res2a_branch2b
 Layer: bn2a_branch2b
 Layer: res2a_branch2c
 Layer: res2a_branch1
 Layer: bn2a_branch2c
 Layer: bn2a_branch1
 Layer: res2b_branch2a
 Layer: bn2b_branch2a
 Layer: res2b_branch2b
 Layer: bn2b_branch2b
 Layer: res2b_branch2c
 Layer: bn2b_branch2c
 Layer: res2c_branch2a
 Layer: bn2c_branch2a
 Layer: res2c_branch2b
 Layer: bn2c_branch2b
 Layer: res2c_branch2c
 Layer: bn2c_branch2c
 Layer: res3a_branch2a
 Layer: bn3a_branch2a
 Layer: res3a_branch2b
 Layer: bn3a_branch2b
 Layer: res3a_branch2c
 Layer: res3a_branch1
 Layer: bn3a_branch2c
 Layer: bn3a_branch1
 Layer: res3b_branch2a
 Layer: bn3b_branch2a
 Layer: res3b_branch2b
 Layer: bn3b_branch2b
 Layer: res3b_branch2c
 Layer: bn3b_branch2c
 Layer: res3c_branch2a
 Layer: bn3c_branch2a
 Layer: res3c_branch2b
 Layer: bn3c_branch2b
 Layer: res3c_branch2c
 Layer: bn3c_branch2c
 Layer: res3d_branch2a
 Layer: bn3d_branch2a
 Layer: res3d_branch2b
 Laye

####  Get a list of trainable layers, i.e., layers that have weights

In [ ]:
# trainable = mrcnn_model.get_trainable_layers()
layers    = [layer for layer in mrcnn_model.keras_model.layers]          # all layer outputs
print('Trn  Idx   Layer Name   \t\t InputNode LayerNames \t\t InputNode LayerShapes')
for idx, layer in enumerate(layers):
    TrnFlg = 'T' if layer.get_weights() else ' '
    all_input_shapes = [node.input_shapes   for node in layer._inbound_nodes][0]
    input_nodes  = [node for node in layer._inbound_nodes]
    all_nodes_layers  = [node.inbound_layers for node in input_nodes]   
#     print(all_input_nodes)
#     all_nodes_layers  = [[node.inbound_layers for node in input_nodes]  for input_nodes in layer._inbound_nodes]  #[node for node in all_input_nodes]]]
    all_input_layer_names = [[layer.name for layer in node_layers] for node_layers in all_nodes_layers][0]        
    str_names = ','.join(map(str, all_input_layer_names)) 
    str_shapes = ','.join(map(str, all_input_shapes)) 
    print(' {} ; {:3d} ; {:25s}; {:30s}; {:30s} '.format( TrnFlg, idx, layer.name,  str_names, str_shapes))

In [ ]:
print()    
for i in range(194,len(layers)):
# for i in [206]:
    layer = layers[i]
    print('====================================================='.format(i))
    print('== Layer {} : {} '.format(i,layer.name))
    print('====================================================='.format(i))
    pp.pprint(layer.__dict__)
    print()
    for j,node in enumerate(layer._inbound_nodes):
        print('== Inbound Node {} ========================================='.format(j))
        print(type(node))
        pp.pprint(node.__dict__)
        node_inlayers_names = [n.name for n in node.inbound_layers]
        print(node_inlayers_names)